This repo is for fasttext model with ichi dataset

In [ ]:
import nltk
nltk.download('stopwords')
!pip install imblearn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%matplotlib inline

import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
import seaborn as sns
from imblearn.over_sampling import SMOTE

#importing data files and preprocessing

In [ ]:
import pandas as pd
df_train = pd.read_csv('./data/train_test_split/ichi/ichi_train.tsv',sep="\t")
df_train.drop(['Title'],axis=1,inplace=True)

In [ ]:
df_test = pd.read_csv('./data/train_test_split/ichi/ichi_test.tsv',sep='\t')
df_test.drop(['Title'],axis=1,inplace=True)
df_test

,Category,Question,Concepts
0,DISE,Hi All! I am new here but have been lurking fo...,NaN
1,SOCL,My girlfriend and i just got through having se...,NaN
2,GOAL,Dr. i have dirty yellow buning eyes since my t...,condition|eyes|HAND
3,SOCL,"Hi, a few nights ago I went to a gay sexclub a...",NaN
4,FAML,my 4 year old is a nightmare. me and my husban...,demanding|fits
...,...,...,...
2995,TRMT,I am definitley having my lap band removed in ...,acid reflux|said|weight
2996,PREG,I am in the TWW again. I was on Femara this mo...,follicles
2997,DISE,I've been advised to try visual routine charts...,NaN
2998,PREG,Just looking for some cycle buddies! I star...,NaN


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
categories = ['DEMO','DISE','FAML','GOAL','PREG','SOCL','TRMT']

In [ ]:
df_train['Question'] = df_train['Question'].map(lambda com : clean_text(com))
df_test['Question'] = df_test['Question'].map(lambda com : clean_text(com))

# import fasttext repo

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git # clone fasttext repo
cd fastText # change directory to fasttext to use the fasttext code


In [ ]:
!make # rbuild the makefile

make: Nothing to be done for 'opt'.


## preprocessing the train and test data to use fasttext 

In [ ]:
df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: '__label__' + x)
df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: '__label__' + x)

In [ ]:
df_test

,Category,Question,Concepts
0,__label__DISE,hi all i am new here but have been lurking for...,NaN
1,__label__SOCL,my girlfriend and i just got through having se...,NaN
2,__label__GOAL,dr i have dirty yellow buning eyes since my te...,condition|eyes|HAND
3,__label__SOCL,hi a few nights ago i went to a gay sexclub an...,NaN
4,__label__FAML,my 4 year old is a nightmare me and my husband...,demanding|fits
...,...,...,...
2995,__label__TRMT,i am definitley having my lap band removed in ...,acid reflux|said|weight
2996,__label__PREG,i am in the tww again i was on femara this mon...,follicles
2997,__label__DISE,i have been advised to try visual routine char...,NaN
2998,__label__PREG,just looking for some cycle buddies i started ...,NaN


In [ ]:
import csv
df_train[['Category', 'Question']].to_csv(r'train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")
df_test[['Category', 'Question']].to_csv(r'test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

# training with the data

In [ ]:
! ./fasttext supervised -input train.txt -output model -lr 1e-1 -epoch 20

Read 1M words
Number of words:  37389
Number of labels: 7
Progress: 100.0% words/sec/thread:  503364 lr:  0.000000 avg.loss:  0.474432 ETA:   0h 0m 0s


# prediction result dumped on txt file

In [ ]:
! ./fasttext predict model.bin test.txt 1  >fastTextResult.txt # storing the prediction on fastTextResult.tx

In [ ]:
testDataCat=df_test['Category'].tolist()
a_file = open("fastTextResult.txt")
resultClass=[]
lines = a_file. readlines()
for line in lines:
  # print(line[9:13])
  resultClass.append(line[:13])# taking upto 13 charecter to avoid \n ie:   '__label__GOAL\n',

# comparing the result

In [ ]:
correctPredict=0
for i in range(len(resultClass)):
  if resultClass[i]==testDataCat[i]:
    correctPredict=correctPredict+1


In [ ]:
correctPredict

1821

In [ ]:
correctPredict/len(resultClass)


0.607